In [1]:
import os
import random
import math
import numpy as np
import scipy.stats

from goes_sharpening_functions import *

/sharedData2/nshakoor/.conda/envs/fires/lib/python3.7/site-packages/pyresample/bilinear/__init__.py:49: UserWarning: XArray and/or zarr not found, XArrayBilinearResampler won't be available.
  warnings.warn("XArray and/or zarr not found, XArrayBilinearResampler won't be available.")
2023-01-01 15:20:23.597144: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2023-01-01 15:20:23.597254: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2023-01-01 15:20:23.597265: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properl

In [2]:
def find_npy_file(path, file, band):
    """
    Returns the string name of another file from the same time for a given GOES band npy file,
    or None if there is no such file in the given directory.
    
    Parameter path: The path where the returned file is located in
    Precondition: path is a string to a directory relative to the current one, with .npy files
    
    Parameter file: A file from the same time the returned file should have
    Precondition: file is a string with the name of the original file, and includes the path
    
    Parameter band: GOES band the returned file should be from
    Precondition: band is a string of length 2
    """
    for x in os.listdir(path):
        if x[19:21] == band and file[23:37] == x[27:41]:
            return x
        
def fire_mask(img07, img14):
    """
    Returns where there are fires.
    """
    m = (330-210)/(340-295)
    b = 210 - m*295
    return (img14) < (m*img07 + b)

def dice_score(x, y):
    numerator = 2*np.count_nonzero(np.logical_and(x, y))
    denominator = np.count_nonzero(x) + np.count_nonzero(y)
    return numerator/denominator

def other_stats(x, y):
    false_positives = np.logical_and(np.logical_not(x), y)
    false_negatives = np.logical_and(x, np.logical_not(y))
    
    #true positive rate
    TPR = np.count_nonzero(np.logical_and(x, y))/np.count_nonzero(x)
    
    #positive predictive value
    try:
        PPV = np.count_nonzero(np.logical_and(x, y))/np.count_nonzero(y)
    except ZeroDivisionError:
        PPV = float('NaN')
        
    return [np.count_nonzero(false_positives), np.count_nonzero(false_negatives), TPR, PPV]

In [3]:
path = '../../goes_files/npy_files/'
pathCM = '../../goes_files/clear_sky_mask/'

filedates = set()
files = []

for file in os.listdir(path):
    filedates.add(file[27:41])

for file in os.listdir(pathCM):
    if file[23:37] in filedates:
        files.append(file)

In [7]:
data = {'control':{}, 'laplace':{}, 'unsharpmask':{}, 'esrgan':{}}

for key in data:
    data[key] = {'num_fires':[], 'dice_score':[], 'TPR':[], 'PPV':[], 'corr_coef':[], 'rmse':[]}

In [5]:
def update_dict(fires, cloud_mask, img07, img14, data):
    for algo in data:
        data07 = eval(algo + "_sharpening_img")(img07)
        data14 = eval(algo + "_sharpening_img")(img14)
        datafires = np.logical_and(fire_mask(data07, data14), np.logical_not(cloud_mask))
        otherstats = other_stats(fires, datafires)
        
        data[algo]['num_fires'].append(np.count_nonzero(datafires))
        data[algo]['dice_score'].append(dice_score(fires, datafires))
        data[algo]['TPR'].append(otherstats[2])
        data[algo]['PPV'].append(otherstats[3])
        data[algo]['corr_coef'].append(scipy.stats.pearsonr((img07-img14).flatten(), (data07-data14).flatten())[0])
        data[algo]['rmse'].append(math.sqrt(np.square((img07-img14) - (data07-data14)).mean()))
        
def results(data, stat):
    print(stat)
    for algo in data:
        print(f'     {algo} {round(np.nanmean(data[algo][stat]), 4)} +/- {round(np.nanstd(data[algo][stat]), 4)}')

In [12]:
num_fires = []
x = 0

while x < 50:
    try:
        CM = random.choice(files)
        files.remove(CM)

        file07 = find_npy_file(path, CM, '07')
        file14 = find_npy_file(path, CM, '14')
        img07 = np.load(path + file07)
        img14 = np.load(path + file14)

        cloud_mask = np.load(pathCM + CM)
        fires = np.logical_and(fire_mask(img07, img14), np.logical_not(cloud_mask))

        if np.count_nonzero(fires) != 0:
            num_fires.append(np.count_nonzero(fires))
            update_dict(fires, cloud_mask, img07, img14, data)

            x += 1
    except:
        pass

/sharedData2/nshakoor/.conda/envs/fires/lib/python3.7/site-packages/skimage/transform/_warps.py:725: RuntimeWarning: All-NaN slice encountered
  min_val = min_func(input_image)
/sharedData2/nshakoor/.conda/envs/fires/lib/python3.7/site-packages/skimage/transform/_warps.py:729: RuntimeWarning: All-NaN slice encountered
  max_val = max_func(input_image)


In [19]:
results(data, 'dice_score')

dice_score
     control 0.6657 +/- 0.2993
     laplace 0.6947 +/- 0.2391
     unsharpmask 0.7083 +/- 0.2543
     esrgan 0.8005 +/- 0.216


In [20]:
results(data, 'corr_coef')

corr_coef
     control 0.9778 +/- 0.0094
     laplace 0.9848 +/- 0.0065
     unsharpmask 0.9833 +/- 0.0071
     esrgan 0.9844 +/- 0.0064


In [21]:
results(data, 'rmse')

rmse
     control 1.3503 +/- 0.5414
     laplace 1.1048 +/- 0.4438
     unsharpmask 1.1612 +/- 0.4644
     esrgan 1.2094 +/- 0.5773
